# Colab: Загрузка и переключение LoRA-адаптеров

**Цель:** загрузить базовую модель и несколько адаптеров, переключаться между ними при генерации.

> Пути к адаптерам укажите свои (Drive, локальный output, Hugging Face Hub).

## 0. Установка

In [ ]:
!pip install -q transformers peft torch accelerate

## 1. Подключение Google Drive (опционально)

Если адаптеры сохранены в Drive.

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

# Пример: адаптеры в Drive
ADAPTER_JSON = "/content/drive/MyDrive/adapters/lora_json"
ADAPTER_CODING = "/content/drive/MyDrive/adapters/lora_coding"
ADAPTER_SUPPORT = "/content/drive/MyDrive/adapters/lora_support"

## 2. Загрузка модели и адаптеров

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

BASE_MODEL = "mistralai/Mistral-7B-v0.1"

# Пути: подставьте свои. Для одного адаптера из peft_tools_hw — outputs_finetome/final
ADAPTER_PATHS = {
    "default": "outputs_finetome/final",  # после peft_tools_hw.ipynb
    # "json": "/content/drive/MyDrive/adapters/lora_json",
    # "coding": "/content/drive/MyDrive/adapters/lora_coding",
    # "support": "/content/drive/MyDrive/adapters/lora_support",
}

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    torch_dtype=torch.float16
)

# Загружаем первый адаптер (обязательно)
first_adapter = list(ADAPTER_PATHS.keys())[0]
first_path = ADAPTER_PATHS[first_adapter]

model = PeftModel.from_pretrained(base_model, first_path, adapter_name=first_adapter)

# Добавляем остальные адаптеры (если есть)
for name, path in ADAPTER_PATHS.items():
    if name != first_adapter:
        try:
            model.load_adapter(path, adapter_name=name)
            print(f"Загружен адаптер: {name}")
        except Exception as e:
            print(f"Пропущен {name}: {e}")

print("Доступные адаптеры:", model.peft_config.keys())

## 3. Генерация с переключением

In [ ]:
def generate(prompt: str, adapter_name: str = None, max_new_tokens: int = 150):
    """Генерация с выбранным адаптером."""
    if adapter_name and adapter_name in model.peft_config:
        model.set_adapter(adapter_name)
        print(f"Адаптер: {adapter_name}")
    
    full_prompt = f"""### Instruction:
{prompt}

### Response:
"""
    
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(out[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

In [ ]:
# Пример с одним адаптером (default)
prompt = "Объясни, что такое рекурсия в Python, в двух предложениях."
print(generate(prompt, adapter_name="default"))

In [ ]:
# Если загружены несколько адаптеров — переключаем
# print(generate("Извлеки данные в JSON", adapter_name="json"))
# print(generate("Напиши функцию сортировки", adapter_name="coding"))
# print(generate("Пользователь не может войти в аккаунт", adapter_name="support"))

## 4. Альтернатива: один адаптер через AutoPeftModelForCausalLM

Если нужен только один адаптер — проще загрузить сразу base+adapter.

In [ ]:
# from peft import AutoPeftModelForCausalLM
# model = AutoPeftModelForCausalLM.from_pretrained(
#     "outputs_finetome/final",
#     device_map="auto",
#     torch_dtype=torch.float16
# )
# tokenizer = AutoTokenizer.from_pretrained("outputs_finetome/final")
# # Дальше model.generate(...) как обычно